#### Define Imports

In [1]:
#%pip install openai

In [2]:
import pandas as pd 
from sklearn.model_selection import train_test_split
import json
import os
import openai
from openai import OpenAI
import time
import sys

sys.path.insert(0,'../')
from environment import env
from environment import helper_functions

#### Define Variables

In [3]:
config = env.env()

# Inputs
client = OpenAI(api_key  = config['gpt_api_key'])
root = config['root']
syn_queries = root+'synthetic_queries.csv'


# Output
train_data = 'endpoint_train_data.jsonl'
val_data = 'endpoint_val_data.jsonl'
test_data = 'endpoint_test_data.jsonl'
model_name = 'nps_model_endpoint'
target = 'endpoint'

#### Read Inputs

In [4]:
synthetic_queries_df = pd.read_csv(syn_queries)

#### Prepare for GPT

In [5]:
train_df, val_df = train_test_split(synthetic_queries_df, test_size=0.2, random_state=42)
train_df, temp_df = train_test_split(synthetic_queries_df, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

helper_functions.save_to_jsonl(train_df, train_data, target)
helper_functions.save_to_jsonl(val_df, val_data, target)
helper_functions.save_to_jsonl(test_df, test_data, target)

#### Fine Tune

In [6]:
train_file =  client.files.create(
  file=open(train_data, "rb"),
  purpose="fine-tune"
)

val_file = client.files.create(
  file=open(val_data, "rb"),
  purpose="fine-tune"
)

train_file_id = train_file.id
val_file_id = val_file.id

In [87]:
fine_tune = client.fine_tuning.jobs.create(
    model="davinci-002",
    training_file=train_file_id,
    validation_file=val_file_id,
    seed = 42,
    suffix = model_name
)
fine_tune_id = fine_tune.id